In [43]:
import sys
import os
root_path = os.path.dirname(os.getcwd()) 
sys.path.append(root_path)

import plotly.express as px

from src import alpaca
from src import plotting

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
### Get account info

account_info = alpaca.get_account_info()

In [45]:
### Get bars

df = alpaca.get_bars("2022-01-01", "2023-01-01")
fig = plotting.bar_plot(df)
fig.show()